In [2]:


import pickle

file = open('//content//embedding_matrix.txt','rb')
embedding_matrix = pickle.load(file)
file.close()


FileNotFoundError: [Errno 2] No such file or directory: '//content//embedding_matrix.txt'

In [3]:

with open('/content/train_dict.txt','rb') as fp:
    train_dict = pickle.load(fp)
    fp.close()
with open('/content/word_to_indx.txt','rb') as fp:
    word_to_indx = pickle.load(fp)
    fp.close()

FileNotFoundError: [Errno 2] No such file or directory: '/content/train_dict.txt'

In [4]:
with open('/content/encode_train.txt', 'rb') as fp:
    encoding_train = pickle.load(fp)
    fp.close()

FileNotFoundError: [Errno 2] No such file or directory: '/content/encode_train.txt'

In [ ]:
from keras import layers
from keras.models import Model,load_model
import numpy as np

In [ ]:
maxlen=32
vocab_size=18060

In [ ]:
input_img_features=layers.Input(shape=(2048,))
inp_img1=layers.Dropout(0.3)(input_img_features)
inp_img2=layers.Dense(256,activation='relu')(inp_img1)

In [ ]:
input_captions=layers.Input(shape=(maxlen,))
inp_cap1=layers.Embedding(input_dim=vocab_size,output_dim=50,mask_zero=True)(input_captions)
inp_cap2=layers.Dropout(0.3)(inp_cap1)
inp_cap3=layers.LSTM(256)(inp_cap2)


In [ ]:
decoder1=layers.add([inp_img2,inp_cap3])
decoder2=layers.Dense(256,activation='relu')(decoder1)
outputs=layers.Dense(vocab_size,activation='softmax')(decoder2)

model=Model(inputs=[input_img_features,input_captions],outputs=outputs)

In [ ]:
model.summary()

Model: "model"
__________________________________________________________________________________________________
 Layer (type)                Output Shape                 Param #   Connected to                  
 input_2 (InputLayer)        [(None, 32)]                 0         []                            
                                                                                                  
 input_1 (InputLayer)        [(None, 2048)]               0         []                            
                                                                                                  
 embedding (Embedding)       (None, 32, 50)               903000    ['input_2[0][0]']             
                                                                                                  
 dropout (Dropout)           (None, 2048)                 0         ['input_1[0][0]']             
                                                                                              

In [ ]:
embedding_matrix[0]

array([ 0.33042  ,  0.24995  , -0.60874  ,  0.10923  ,  0.036372 ,
        0.151    , -0.55083  , -0.074239 , -0.092307 , -0.32821  ,
        0.09598  , -0.82269  , -0.36717  , -0.67009  ,  0.42909  ,
        0.016496 , -0.23573  ,  0.12864  , -1.0953   ,  0.43334  ,
        0.57067  , -0.1036   ,  0.20422  ,  0.078308 , -0.42795  ,
       -1.7984   , -0.27865  ,  0.11954  , -0.12689  ,  0.031744 ,
        3.8631   , -0.17786  , -0.082434 , -0.62698  ,  0.26497  ,
       -0.057185 , -0.073521 ,  0.46103  ,  0.30862  ,  0.12498  ,
       -0.48609  , -0.0080272,  0.031184 , -0.36576  , -0.42699  ,
        0.42164  , -0.11666  , -0.50703  , -0.027273 , -0.53285  ])

In [ ]:
model.layers[2].set_weights([embedding_matrix])
model.layers[2].trainable=False

In [ ]:
model.compile(loss='categorical_crossentropy',optimizer='adam')

In [ ]:
epochs=20
batch_size=3
steps=len(train_dict)//batch_size


In [ ]:
from keras.preprocessing.sequence import pad_sequences
from keras.utils import to_categorical


In [ ]:
def data_generator(train_dict,encoding_train,word_to_indx,max_len,batch_size):
    X1,X2,y=[],[],[]
    n=0
    while True:
        for key,desc_list in train_dict.items():
            n+=1
            photo=encoding_train.get(key)
            if photo is None:
              print(f"Warning: Key '{key+'.jpg'}' not found in encoding_train. Skipping...")
              continue  # Skip to the next iteration if photo is not found

            for desc in desc_list:
                seq=[word_to_indx[word] for word in desc.split() if word in word_to_indx]
                for i in range(len(seq)):
                    xi=seq[0:i]
                    yi=seq[i]
                    xi=pad_sequences([xi],maxlen=maxlen,value=0,padding='post')[0]
                    yi=to_categorical([yi],num_classes=vocab_size)[0]
                    X1.append(photo)
                    X2.append(xi)
                    y.append(yi)
            if(n==batch_size):
              yield([[np.array(X1),np.array(X2)],np.array(y)])
              X1,X2,y=[],[],[]
              n=0



In [ ]:

def train():
  for i in range(epochs):
    generator=data_generator(train_dict,encoding_train,word_to_indx,maxlen,batch_size)
    model.fit_generator(generator,steps_per_epoch=steps,epochs=1,verbose=1)
    model.save('/content/model_weights/model_'+str(i)+'.h5')

In [ ]:
train()

<ipython-input-40-185f8dbe0dc5>:4: UserWarning: `Model.fit_generator` is deprecated and will be removed in a future version. Please use `Model.fit`, which supports generators.
  model.fit_generator(generator,steps_per_epoch=steps,epochs=1,verbose=1)


  59/3333 [..............................] - ETA: 7:01 - loss: 3.8285

KeyboardInterrupt: 

In [1]:
model=load_model('/content/model_weights/model_19.h5')

NameError: name 'load_model' is not defined